In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


### Introduction
This notebook will build a Bidirectional LSTM network using keras to solve classification sentiment problem for movie reviews.


Import **libraries**, import **custom scripts** and define **constants**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout, SpatialDropout1D 
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow.keras.backend as K

from sklearn.model_selection import train_test_split

import re


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# import all our functions
import os,sys
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks/ml_training/app_predict/')
from src import preprocessing


In [ ]:
#definition constants
RANDOM_STATE = 11
TEST_SIZE = 0.15


#### Loading the data and applying the preprocessing

In [ ]:
# import & display data
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ml_training/data/IMDB_Dataset.csv')
data['sentiment'] = data['sentiment'].replace({'positive' : 1, 'negative' : 0})
data = data.drop_duplicates()
data['review'] = data['review'].apply(lambda x: preprocessing.preprocessing_text(x))
data.head()


,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. the filming t...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there's a family where a little boy ...,0
4,"petter mattei's ""love in the time of money"" is...",1


#### Split the data for the training, the testing and the validation datasets

In [ ]:
X = data.review
y = data.sentiment
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=TEST_SIZE, 
                                                    random_state=RANDOM_STATE, 
                                                    stratify = y)

X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                    y_train,
                                                    test_size=TEST_SIZE, 
                                                    random_state=RANDOM_STATE, 
                                                    stratify = y_train)


### Preprocessing Data

In [ ]:
max_features = 50000
max_len = 1000


tokenizer = Tokenizer(num_words=max_features, oov_token='unk')

# only fit on train
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)


X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)


Let's use the **Wiki News FastText Embeddings**

In [ ]:
EMBEDDING_FILE = '/content/drive/My Drive/Colab Notebooks/ml_training/app_predict/notebooks/wiki-news-300d-1M.vec/wiki-news-300d-1M.vec'

def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))

embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


### Let's define the function for building and training model

In [ ]:
#function for f1 metric
def get_f1(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val


def build_model(dropout_sp = 0.2,
                dropout=0.5,
                lstm_out=64,
                units=32,
                activation='relu',   
                learning_rate=1e-3, 
                weights=None ):
    model = Sequential()
    model.add(SpatialDropout1D(dropout_sp))
    model.add(Embedding(max_features, embed_size, input_length=max_len, weights=weights ))
    model.add(Dropout(dropout))
    model.add(Bidirectional(LSTM(lstm_out)))
    model.add(Dropout(dropout))
    model.add(Dense(units, activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(lr=learning_rate)

    model.compile(optimizer, 'binary_crossentropy', metrics=[get_f1])
    return model


def train_model(lstm_out=64, 
                dropout=0.5, 
                weights = None,
                optimizer='adam',
                batch_size=32,
                epochs=20,
                activation='relu', 
                units=32, 
                log_dir='logs/bilstm',
                filepath='model.hdf5'):
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    early_stop = EarlyStopping(monitor='val_loss', 
                               patience=3)
    cp_callback = ModelCheckpoint(filepath=filepath,
                                  save_best_only=True,
                                  verbose=1)
    
    model = build_model(lstm_out, dropout, optimizer, activation, units, weights)
    model.summary()
    history = model.fit(X_train_pad, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=2, 
                        callbacks=[tensorboard_callback, early_stop, cp_callback],                    
                        validation_data=(X_val_pad, y_val))
    return history
    

    

### Train different models

Result of exploring let's save to Dataframe

In [ ]:
models_research_df = pd.DataFrame(columns=['Model',
                                           'loss',
                                           'f1_score',
                                           'val_loss',
                                           'val_f1'], )
results = {}
# add data to result research dataframe
def add_data_df(table_df_for_add, model_name, history_research):
    index = history_research['val_loss'].index(min(history_research['val_loss']))
    model_research = (model_name, history_research['loss'][index], history_research['get_f1'][index], history_research['val_loss'][index], history_research['val_get_f1'][index])
    if any([list(row.values ) == list(model_research) for _, row in table_df_for_add.iterrows()]):
        return
    table_df_for_add.loc[len(table_df_for_add)] = model_research


### Model_1
The model with default params for this model with `dropout=0` 

In [ ]:
history_1 = train_model(filepath='Model_1_{epoch:02d}-{val_get_f1:.2f}.hdf5',
           log_dir='logs/bilstm/model_1',
           dropout=0)
results['model_1'] = history_1

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 300)         15000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               186880    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 15,191,041
Trainable params: 15,191,041
Non-trainable params: 0
____________________________________________

### Model_2
The model with `dropout=0` and pretrained Embeddings

In [ ]:
history_2 = train_model(filepath='Model_2_{epoch:02d}-{val_get_f1:.2f}.hdf5',
           log_dir='logs/bilstm/model_2',
           dropout=0, 
           weights=[embedding_matrix])
results['model_2'] = history_2

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 300)         15000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               186880    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 15,191,041
Trainable params: 15,191,041
Non-trainable params: 0
__________________________________________

### Result of exploring models

In [ ]:
for result in results:
    add_data_df(models_research_df, result, results.get(result).history)

In [ ]:
models_research_df

,Model,loss,f1_score,val_loss,val_f1
0,model_1,0.235243,0.905369,0.278430,0.883599
1,model_2,0.207167,0.918736,0.254204,0.901164


The model with pretrained embeddings has best metric f1